In [1]:
import gresearch_crypto

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/g-research-crypto-forecasting/example_sample_submission.csv
/kaggle/input/g-research-crypto-forecasting/asset_details.csv
/kaggle/input/g-research-crypto-forecasting/example_test.csv
/kaggle/input/g-research-crypto-forecasting/train.csv
/kaggle/input/g-research-crypto-forecasting/supplemental_train.csv
/kaggle/input/g-research-crypto-forecasting/gresearch_crypto/competition.cpython-37m-x86_64-linux-gnu.so
/kaggle/input/g-research-crypto-forecasting/gresearch_crypto/__init__.py
/kaggle/input/modeljan152022/ExpModel_1.hdf5
/kaggle/input/scaler3/scaler-3.save


In [3]:
#import warnings
#warnings.filterwarnings('ignore')

import datatable as dt
import gc
# import cudf
import pandas as pd
import numpy as np
# import cupy as cp
# import janestreet
#import xgboost as xgb
#from hyperopt import hp, fmin, tpe, Trials
#from hyperopt.pyll.base import scope
from sklearn.metrics import mean_squared_error, roc_auc_score, roc_curve
#from sklearn.model_selection import GroupKFold
#from sklearn.preprocessing import StandardScaler
#from sklearn.pipeline import Pipeline
#import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
#from joblib import dump, load

import tensorflow as tf
#tf.random.set_seed(42)
import tensorflow.keras.backend as K
import tensorflow.keras.layers as layers

2022-02-01 13:19:53.561361: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2022-02-01 13:19:53.561468: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
%%time
crypto_df = pd.read_csv('../input/g-research-crypto-forecasting/train.csv')
suppl_df = pd.read_csv('../input/g-research-crypto-forecasting/supplemental_train.csv')

asset_details = pd.read_csv('../input/g-research-crypto-forecasting/asset_details.csv')

train_data_start = 1555079640
train_data_end = 1623542400

data_1 = crypto_df.loc[(crypto_df['timestamp'] >= train_data_start) & (crypto_df['timestamp'] <= train_data_end)]
data_2 = suppl_df.loc[suppl_df['timestamp'] > train_data_end]

total_df = pd.concat([data_1, data_2])

total_df = total_df.merge(asset_details, on='Asset_ID')

weights = asset_details.sort_values(by = ['Asset_ID'], ascending = True)[['Weight']].T

CPU times: user 42.7 s, sys: 6 s, total: 48.7 s
Wall time: 1min 13s


In [5]:
df = total_df

bnb = df[df['Asset_ID'] == 0].set_index("timestamp")
btc = df[df['Asset_ID'] == 1].set_index("timestamp")
bch = df[df['Asset_ID'] == 2].set_index("timestamp")
ada = df[df['Asset_ID'] == 3].set_index("timestamp")
doge = df[df['Asset_ID'] == 4].set_index("timestamp")
eos = df[df['Asset_ID'] == 5].set_index("timestamp")
eth = df[df['Asset_ID'] == 6].set_index("timestamp")
etc = df[df['Asset_ID'] == 7].set_index("timestamp")
miota = df[df['Asset_ID'] == 8].set_index("timestamp")
ltc = df[df['Asset_ID'] == 9].set_index("timestamp")
mkr = df[df['Asset_ID'] == 10].set_index("timestamp")
xmr = df[df['Asset_ID'] == 11].set_index("timestamp")
xlm = df[df['Asset_ID'] == 12].set_index("timestamp")
trx = df[df['Asset_ID'] == 13].set_index("timestamp")

In [6]:
bnb_last_month = bnb[['Open','High','Low','Close','VWAP']].loc[1629503040 : ]
bnb_mean = pd.DataFrame(bnb_last_month.mean()).T

btc_last_month = btc[['Open','High','Low','Close','VWAP']].loc[1629503040 : ]
btc_mean = pd.DataFrame(btc_last_month.mean()).T

bch_last_month = bch[['Open','High','Low','Close','VWAP']].loc[1629503040 : ]
bch_mean = pd.DataFrame(bch_last_month.mean()).T

ada_last_month = ada[['Open','High','Low','Close','VWAP']].loc[1629503040 : ]
ada_mean = pd.DataFrame(ada_last_month.mean()).T

doge_last_month = doge[['Open','High','Low','Close','VWAP']].loc[1629503040 : ]
doge_mean = pd.DataFrame(doge_last_month.mean()).T

eos_last_month = eos[['Open','High','Low','Close','VWAP']].loc[1629503040 : ]
eos_mean = pd.DataFrame(eos_last_month.mean()).T

eth_last_month = eth[['Open','High','Low','Close','VWAP']].loc[1629503040 : ]
eth_mean = pd.DataFrame(eth_last_month.mean()).T

etc_last_month = etc[['Open','High','Low','Close','VWAP']].loc[1629503040 : ]
etc_mean = pd.DataFrame(etc_last_month.mean()).T

miota_last_month = miota[['Open','High','Low','Close','VWAP']].loc[1629503040 : ]
miota_mean = pd.DataFrame(miota_last_month.mean()).T

ltc_last_month = ltc[['Open','High','Low','Close','VWAP']].loc[1629503040 : ]
ltc_mean = pd.DataFrame(ltc_last_month.mean()).T

mkr_last_month = mkr[['Open','High','Low','Close','VWAP']].loc[1629503040 : ]
mkr_mean = pd.DataFrame(mkr_last_month.mean()).T

xmr_last_month = xmr[['Open','High','Low','Close','VWAP']].loc[1629503040 : ]
xmr_mean = pd.DataFrame(xmr_last_month.mean()).T

xlm_last_month = xlm[['Open','High','Low','Close','VWAP']].loc[1629503040 : ]
xlm_mean = pd.DataFrame(xlm_last_month.mean()).T

trx_last_month = trx[['Open','High','Low','Close','VWAP']].loc[1629503040 : ]
trx_mean = pd.DataFrame(trx_last_month.mean()).T

means = pd.concat([bnb_mean, 
                  btc_mean, 
                  bch_mean, 
                  ada_mean, 
                  doge_mean, 
                  eos_mean, 
                  eth_mean, 
                  etc_mean, 
                  miota_mean, 
                  ltc_mean, 
                  mkr_mean, 
                  xmr_mean, 
                  xlm_mean, 
                  trx_mean])

means.reset_index(drop=True, inplace=True)

In [7]:
data = pd.concat([bnb[['Open','High','Low','Close','VWAP','Target']], 
                  btc[['Open','High','Low','Close','VWAP','Target']], 
                  bch[['Open','High','Low','Close','VWAP','Target']], 
                  ada[['Open','High','Low','Close','VWAP','Target']], 
                  doge[['Open','High','Low','Close','VWAP','Target']], 
                  eos[['Open','High','Low','Close','VWAP','Target']], 
                  eth[['Open','High','Low','Close','VWAP','Target']], 
                  etc[['Open','High','Low','Close','VWAP','Target']], 
                  miota[['Open','High','Low','Close','VWAP','Target']], 
                  ltc[['Open','High','Low','Close','VWAP','Target']], 
                  mkr[['Open','High','Low','Close','VWAP','Target']], 
                  xmr[['Open','High','Low','Close','VWAP','Target']], 
                  xlm[['Open','High','Low','Close','VWAP','Target']], 
                  trx[['Open','High','Low','Close','VWAP','Target']]], axis=1)

In [8]:
data.columns = ['Open-0', 'High-0', 'Low-0', 'Close-0', 'VWAP-0','Target_0',
                 'Open-1', 'High-1', 'Low-1', 'Close-1', 'VWAP-1','Target_1',
                 'Open-2', 'High-2', 'Low-2', 'Close-2', 'VWAP-2','Target_2',
                 'Open-3', 'High-3', 'Low-3', 'Close-3', 'VWAP-3','Target_3',
                 'Open-4', 'High-4', 'Low-4', 'Close-4', 'VWAP-4','Target_4',
                 'Open-5', 'High-5', 'Low-5', 'Close-5', 'VWAP-5','Target_5',
                 'Open-6', 'High-6', 'Low-6', 'Close-6', 'VWAP-6','Target_6',
                 'Open-7', 'High-7', 'Low-7', 'Close-7', 'VWAP-7','Target_7',
                 'Open-8', 'High-8', 'Low-8', 'Close-8', 'VWAP-8','Target_8',
                 'Open-9', 'High-9', 'Low-9', 'Close-9', 'VWAP-9','Target_9',
                 'Open-10', 'High-10', 'Low-10', 'Close-10', 'VWAP-10','Target_10',
                 'Open-11', 'High-11', 'Low-11', 'Close-11', 'VWAP-11','Target_11',
                 'Open-12', 'High-12', 'Low-12', 'Close-12', 'VWAP-12','Target_12',
                 'Open-13', 'High-13', 'Low-13', 'Close-13', 'VWAP-13','Target_13']

In [9]:
data = data.replace([np.inf, -np.inf], np.nan).dropna(how="any")

In [10]:
X_data = data[['Open-0', 'High-0', 'Low-0', 'Close-0', 'VWAP-0',
                 'Open-1', 'High-1', 'Low-1', 'Close-1', 'VWAP-1',
                 'Open-2', 'High-2', 'Low-2', 'Close-2', 'VWAP-2',
                 'Open-3', 'High-3', 'Low-3', 'Close-3', 'VWAP-3',
                 'Open-4', 'High-4', 'Low-4', 'Close-4', 'VWAP-4',
                 'Open-5', 'High-5', 'Low-5', 'Close-5', 'VWAP-5',
                 'Open-6', 'High-6', 'Low-6', 'Close-6', 'VWAP-6',
                 'Open-7', 'High-7', 'Low-7', 'Close-7', 'VWAP-7',
                 'Open-8', 'High-8', 'Low-8', 'Close-8', 'VWAP-8',
                 'Open-9', 'High-9', 'Low-9', 'Close-9', 'VWAP-9',
                 'Open-10', 'High-10', 'Low-10', 'Close-10', 'VWAP-10',
                 'Open-11', 'High-11', 'Low-11', 'Close-11', 'VWAP-11',
                 'Open-12', 'High-12', 'Low-12', 'Close-12', 'VWAP-12',
                 'Open-13', 'High-13', 'Low-13', 'Close-13', 'VWAP-13']]

In [11]:
targets = data[['Target_0','Target_1','Target_2','Target_3','Target_4','Target_5','Target_6','Target_7',
                     'Target_8','Target_9','Target_10','Target_11','Target_12','Target_13']]

In [12]:
train = X_data.loc[train_data_start : ]
y_train = targets.loc[train_data_start : ]

#test = X_data.loc[train_data_end : ]
#y_test = targets.loc[train_data_end : ]

In [13]:
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

scaler.fit(train)

X_train = pd.DataFrame(scaler.transform(train), columns = train.columns)

In [14]:
import os, gc
from sklearn.metrics import mean_squared_error, roc_auc_score, roc_curve
from tqdm.notebook import tqdm

import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow.keras.layers as layers

In [15]:
def correlation_coefficient_loss(y_true, y_pred):
    x = tf.cast(y_true, tf.float64)
    y = tf.cast(y_pred, tf.float64)
    w = tf.cast(weights, tf.float64)
    
    mean_x = K.mean(x, axis = 0)
    w_mean_x = K.sum(tf.multiply(mean_x, w)) / K.sum(w) #mean_a
    
    mean_y = K.mean(y, axis = 0)
    w_mean_y = K.sum(tf.multiply(mean_y, w)) / K.sum(w) #mean_b
    
    mean_xy = K.mean(tf.multiply(x, y), axis = 0)
    cov = K.sum(tf.multiply(mean_xy, w)) / K.sum(w) - w_mean_x * w_mean_y #cov
    
    xm, ym = x - w_mean_x, y - w_mean_y
    
    mean_xx = K.mean(K.square(xm), axis = 0)
    var_x = K.sum(tf.multiply(mean_xx, w)) / K.sum(w)
    
    mean_yy = K.mean(K.square(ym), axis = 0)
    var_y = K.sum(tf.multiply(mean_yy, w)) / K.sum(w)
    
    r = cov / K.sqrt(var_x * var_y)
    r = K.maximum(K.minimum(r, 1.0), -1.0)
    return 1 - K.square(r)

In [16]:
SEED = 1977
tf.random.set_seed(SEED)
np.random.seed(SEED)

In [17]:
def create_mlp(num_columns, num_labels, hidden_units, dropout_rates, learning_rate):
    
    inp = tf.keras.layers.Input(shape = (num_columns, ))
    x = tf.keras.layers.BatchNormalization()(inp)
    x = tf.keras.layers.Dropout(dropout_rates[0])(x)
    for i in range(len(hidden_units)): 
        x = tf.keras.layers.Dense(hidden_units[i])(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation(tf.keras.activations.swish)(x)
        x = tf.keras.layers.Dropout(dropout_rates[i+1])(x)    
        
    out = tf.keras.layers.Dense(num_labels)(x)
    #out = tf.keras.layers.Activation('linear')(x)
    
    model = tf.keras.models.Model(inputs = inp, outputs = out)
    model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate),
                  loss = correlation_coefficient_loss, 
                  metrics = tf.keras.metrics.RootMeanSquaredError(name="root_mean_squared_error", dtype=None), 
                 )
    
    return model

In [18]:
%%time

batch_size = 10000
hidden_units = [30, 50, 30]
dropout_rates = [0.2, 0.2, 0.2, 0.2] #one more than layers
learning_rate = 1e-3
    
model = create_mlp(X_train.shape[1], y_train.shape[1], hidden_units, dropout_rates, learning_rate)
model.summary()
    
model.fit(X_train, y_train, epochs = 200, batch_size = batch_size, callbacks = None, verbose = 1)


2022-02-01 13:21:22.302854: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-02-01 13:21:22.306033: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2022-02-01 13:21:22.306065: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2022-02-01 13:21:22.306094: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (33161cbd5dbe): /proc/driver/nvidia/version does not exist
2022-02-01 13:21:22.307178: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operation

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 70)]              0         
_________________________________________________________________
batch_normalization (BatchNo (None, 70)                280       
_________________________________________________________________
dropout (Dropout)            (None, 70)                0         
_________________________________________________________________
dense (Dense)                (None, 30)                2130      
_________________________________________________________________
batch_normalization_1 (Batch (None, 30)                120       
_________________________________________________________________
activation (Activation)      (None, 30)                0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 30)                0     

2022-02-01 13:21:22.921848: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 311639440 exceeds 10% of free system memory.
2022-02-01 13:21:23.282847: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-02-01 13:21:23.296488: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2199995000 Hz


Epoch 1/200
56/56 [==============================] - 4s 43ms/step - loss: 1.0000 - root_mean_squared_error: 0.7799
Epoch 2/200
56/56 [==============================] - 2s 44ms/step - loss: 1.0000 - root_mean_squared_error: 0.8304
Epoch 3/200
56/56 [==============================] - 2s 44ms/step - loss: 1.0000 - root_mean_squared_error: 0.9004
Epoch 4/200
56/56 [==============================] - 2s 43ms/step - loss: 0.9999 - root_mean_squared_error: 1.0101
Epoch 5/200
56/56 [==============================] - 2s 43ms/step - loss: 0.9999 - root_mean_squared_error: 1.1058
Epoch 6/200
56/56 [==============================] - 2s 42ms/step - loss: 0.9998 - root_mean_squared_error: 1.2061
Epoch 7/200
56/56 [==============================] - 2s 43ms/step - loss: 0.9997 - root_mean_squared_error: 1.2363
Epoch 8/200
56/56 [==============================] - 2s 44ms/step - loss: 0.9997 - root_mean_squared_error: 1.2640
Epoch 9/200
56/56 [==============================] - 2s 43ms/step - loss: 0.9996

In [19]:
env = gresearch_crypto.make_env()

In [20]:
iter_test = env.iter_test()

In [21]:
for (test_df, sample_prediction_df) in iter_test:
    sample = test_df[['Asset_ID','Open','High','Low','Close','VWAP']]
    sample = sample.set_index("Asset_ID")
    
    for i in range(14):
        if i not in sample.index:
            sample = sample.append(means.iloc[i])
    
    sample = sample.sort_values(by = ['Asset_ID'])
    x_test = sample.stack().to_frame().T
    x_test_scaled = pd.DataFrame(scaler.transform(x_test), columns = train.columns)
    
    y_pred = model.predict(x_test_scaled)
    
    for j,row in test_df.iterrows():
         sample_prediction_df.loc[sample_prediction_df['row_id'] == row['row_id'], 'Target'] = y_pred[0][int(row['Asset_ID'])]
    env.predict(sample_prediction_df)

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
